In [9]:
import re
import os
import glob
from bs4 import BeautifulSoup
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import *
stemmer = PorterStemmer()

def review_to_words(review):
    text = BeautifulSoup(review, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Convert to lower case
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
    words = [PorterStemmer().stem(w) for w in words] # stem
    
    return words


import pickle

cache_dir = os.path.join("./cache", "fake_news_analysis")  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists

def preprocess_data(data_train,data_valid, data_test, labels_train, labels_valid, labels_test,
                    cache_dir=cache_dir, cache_file="preprocessed_data.pkl"):
    """Convert each review to words; read from cache if available."""

    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # Preprocess training and test data to obtain words for each review
        #words_train = list(map(review_to_words, data_train))
        #words_test = list(map(review_to_words, data_test))
        words_train = [review_to_words(data) for data in data_train]
        words_valid = [review_to_words(data) for data in data_valid]
        words_test = [review_to_words(data) for data in data_test]
        
        
        # Write to cache file for future runs
        if cache_file is not None:
            cache_data = dict(words_train=words_train, words_valid = words_valid, words_test=words_test,
                              labels_train=labels_train, labels_valid = labels_valid, labels_test=labels_test)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        words_train, words_valid, words_test, labels_train, labels_valid, labels_test = (cache_data['words_train'], cache_data['words_valid'],
                cache_data['words_test'], cache_data['labels_train'], cache_data['labels_valid'], cache_data['labels_test'])
    
    return words_train, words_valid, words_test, labels_train, labels_valid, labels_test


import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.externals import joblib
# joblib is an enhanced version of pickle that is more efficient for storing NumPy arrays

def extract_BoW_features(words_train, words_valid, words_test, vocabulary_size=3200,
                         cache_dir=cache_dir, cache_file="bow_features.pkl"):
    """Extract Bag-of-Words for a given set of documents, already preprocessed into words."""
    
    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = joblib.load(f)
            print("Read features from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # Fit a vectorizer to training documents and use it to transform them
        # NOTE: Training documents have already been preprocessed and tokenized into words;
        #       pass in dummy functions to skip those steps, e.g. preprocessor=lambda x: x
        vectorizer = CountVectorizer(max_features=vocabulary_size,
                preprocessor=lambda x: x, tokenizer=lambda x: x)  # already preprocessed
        features_train = vectorizer.fit_transform(words_train).toarray()
        features_valid =  vectorizer.fit_transform(words_valid).toarray()
        # Apply the same vectorizer to transform the test documents (ignore unknown words)
        features_test = vectorizer.transform(words_test).toarray()
        # NOTE: Remember to convert the features using .toarray() for a compact representation
        
        # Write to cache file for future runs (store vocabulary as well)
        if cache_file is not None:
            vocabulary = vectorizer.vocabulary_
            cache_data = dict(features_train=features_train, features_valid=features_valid, features_test=features_test,
                             vocabulary=vocabulary)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                joblib.dump(cache_data, f)
            print("Wrote features to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        features_train, features_valid, features_test, vocabulary = (cache_data['features_train'], cache_data['features_valid'],
                cache_data['features_test'], cache_data['vocabulary'])
    
    # Return both the extracted features as well as the vocabulary
    return features_train, features_valid, features_test, vocabulary

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
def labelToBinary(label): 
    if(label in ["pants-fire", "FALSE", "false" ]):
        return 0
    elif(label in ["TRUE", "true", "barely-true", "half-true", "mostly-true" ]):
        return 1

def mapLablesToNumbers(labels):
    y_label_dict = {"pants-fire" : 0, "FALSE" : 1, "barely-true" : 2, "half-true" : 3, "mostly-true" : 4, "TRUE" : 5, "false": 1, "true": 5}
    mapped_data = labels.apply(lambda x: labelToBinary(x))
    return mapped_data
    

In [11]:
import pandas as pd
columns = ["file","result","news","type","person","desingation","state","party","PF","F","BT","MT","KT","context"]
train_complete_data = pd.read_csv(os.path.join("./data", "train.tsv"), sep = "\t", names = columns)
valid_complete_data = pd.read_csv(os.path.join("./data", "valid.tsv"), sep='\t', names = columns)
test_complete_data = pd.read_csv(os.path.join("./data", "test.tsv"), sep='\t', names = columns)
train_data = train_complete_data["news"]
train_label = mapLablesToNumbers(train_complete_data["result"])
valid_data = valid_complete_data["news"]
valid_label = mapLablesToNumbers(valid_complete_data["result"])
test_data = test_complete_data["news"]
test_label = mapLablesToNumbers(test_complete_data["result"])


print(len(train_data), "train data - Before")
print(len(train_label), "train label - Before")

print(len(valid_data), "valid data - Before")
print(len(valid_label), "valid label - Before")

print(len(test_data), "test data - Before")
print(len(test_label), "test label - Before")

train_data = pd.concat([train_data,valid_data], axis = 0)
train_label = pd.concat([train_label, valid_label], axis = 0)

print(len(train_data), " Train data  - After")
print(len(train_label), "Train label - After")
print(len(valid_data), "valid data - After")
print(len(valid_label), "valid label - After")

10240 train data - Before
10240 train label - Before
1284 valid data - Before
1284 valid label - Before
1267 test data - Before
1267 test label - Before
11524  Train data  - After
11524 Train label - After
1284 valid data - After
1284 valid label - After


In [12]:
train_data, valid_data, test_data, train_label, valid_label, test_label = preprocess_data(train_data, valid_data, test_data, train_label, valid_label, test_label)
train_data, valid_data, test_data, vocabulary = extract_BoW_features(train_data, valid_data, test_data)
valid_data = train_data[10240:]
train_data = train_data[:10240]
valid_label = train_label[10240:]
train_label = train_label[:10240]

print(len(train_data), " Train data  - After")
print(len(valid_data), "valid data - After")
print(len(valid_label), "valid label - After")
print(len(train_label), "Train label - After")

train_data, valid_data, test_data = (pd.DataFrame(train_data), pd.DataFrame(valid_data), pd.DataFrame(test_data))
train_label, valid_label, test_label = (pd.DataFrame(train_label[:].values), pd.DataFrame(valid_label[:].values), pd.DataFrame(test_label[:].values))

train_label 

Wrote preprocessed data to cache file: preprocessed_data.pkl
Wrote features to cache file: bow_features.pkl
10240  Train data  - After
1284 valid data - After
1284 valid label - After
10240 Train label - After


,0
0,0
1,1
2,1
3,0
4,1
5,1
6,1
7,1
8,1
9,1


In [ ]:
test_data

In [13]:
data_dir = './data/xgboost'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    
# First, save the test data to test.csv in the data_dir directory. Note that we do not save the associated ground truth
# labels, instead we will use them later to compare with our model output.
print(len(test_data))
pd.DataFrame(test_data).to_csv(os.path.join(data_dir, 'test.csv'), header = False, index = False)
# TODO: Save the training and validation data to train.csv and validation.csv in the data_dir directory.
#       Make sure that the files you create are in the correct format.

# Solution:
#
#

pd.concat([valid_label, valid_data], axis=1).to_csv(os.path.join(data_dir, 'validation.csv'), header = False, index = False)
pd.concat([train_label, train_data], axis=1).to_csv(os.path.join(data_dir, 'train.csv'), header = False, index = False)



1267


In [14]:
import sagemaker

session = sagemaker.Session() # Store the current SageMaker session

# S3 prefix (which folder will we use)
prefix = 'fake-xgboost'

# TODO: Upload the test.csv, train.csv and validation.csv files which are contained in data_dir to S3 using sess.upload_data().
test_location = None
val_location = None
train_location = None

# Solution:
test_location = session.upload_data(os.path.join(data_dir, 'test.csv'), key_prefix=prefix)
val_location = session.upload_data(os.path.join(data_dir, 'validation.csv'), key_prefix=prefix)
train_location = session.upload_data(os.path.join(data_dir, 'train.csv'), key_prefix=prefix)

In [15]:
# We need to retrieve the location of the container which is provided by Amazon for using XGBoost.
# As a matter of convenience, the training and inference code both use the same container.
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker import get_execution_role

container = get_image_uri(session.boto_region_name, 'xgboost','0.90-1')
role = get_execution_role()

In [17]:
# TODO: Create a SageMaker estimator using the container location determined in the previous cell.
#       It is recommended that you use a single training instance of type ml.m4.xlarge. It is also
#       recommended that you use 's3://{}/{}/output'.format(session.default_bucket(), prefix) as the
#       output path.

xgb = None

# Solution:
xgb = sagemaker.estimator.Estimator(container, # The location of the container we wish to use
                                    role,                                    # What is our current IAM Role
                                    train_instance_count=1,                  # How many compute instances
                                    train_instance_type='ml.m4.xlarge',      # What kind of compute instances
                                    output_path='s3://{}/{}/output'.format(session.default_bucket(), prefix),
                                    sagemaker_session=session)

# TODO: Set the XGBoost hyperparameters in the xgb object. Don't forget that in this case we have a binary
#       label so we should be using the 'binary:logistic' objective.

# Solution:
xgb.set_hyperparameters(max_depth=5,
                        eta=0.6,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        early_stopping_rounds=10,
                        eval_metric = 'rmse',
                        num_round=500)

s3_input_train = sagemaker.s3_input(s3_data=train_location, content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data=val_location, content_type='csv')

In [18]:
xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})

2020-05-08 04:01:19 Starting - Starting the training job...
2020-05-08 04:01:21 Starting - Launching requested ML instances......
2020-05-08 04:02:31 Starting - Preparing the instances for training......
2020-05-08 04:03:33 Downloading - Downloading input data...
2020-05-08 04:04:12 Training - Downloading the training image...
2020-05-08 04:04:33 Training - Training image download completed. Training in progress..INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter eval_metric value rmse to Json.
Returning the value itself
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','

In [19]:
# TODO: Create a transformer object from the trained model. Using an instance count of 1 and an instance type of ml.m4.xlarge
#       should be more than enough.
xgb_transformer = xgb.transformer(instance_count = 1, instance_type = 'ml.m4.xlarge')

# TODO: Start the transform job. Make sure to specify the content type and the split type of the test data.
xgb_transformer.transform(test_location, content_type='text/csv', split_type='Line')

In [20]:
xgb_transformer.wait()

......................[2020-05-08 05:50:59 +0000] [15] [INFO] Starting gunicorn 19.10.0
[2020-05-08 05:50:59 +0000] [15] [INFO] Listening at: unix:/tmp/gunicorn.sock (15)
[2020-05-08 05:50:59 +0000] [15] [INFO] Using worker: gevent
[2020-05-08 05:50:59 +0000] [22] [INFO] Booting worker with pid: 22
[2020-05-08 05:50:59 +0000] [23] [INFO] Booting worker with pid: 23
[2020-05-08 05:50:59 +0000] [24] [INFO] Booting worker with pid: 24
[2020-05-08 05:50:59 +0000] [28] [INFO] Booting worker with pid: 28

[2020-05-08:05:51:24:INFO] No GPUs detected (normal if no gpus installed)
169.254.255.130 - - [08/May/2020:05:51:24 +0000] "GET /ping HTTP/1.1" 200 0 "-" "Go-http-client/1.1"
169.254.255.130 - - [08/May/2020:05:51:24 +0000] "GET /execution-parameters HTTP/1.1" 200 84 "-" "Go-http-client/1.1"
[2020-05-08:05:51:24:INFO] No GPUs detected (normal if no gpus installed)
169.254.255.130 - - [08/May/2020:05:51:24 +0000] "GET /ping HTTP/1.1" 200 0 "-" "Go-http-client/1.1"
169.254.255.130 - - [08/May

In [21]:
!aws s3 cp --recursive $xgb_transformer.output_path $data_dir

download: s3://sagemaker-us-east-1-024597597485/sagemaker-xgboost-2020-05-08-05-47-29-616/test.csv.out to data/xgboost/test.csv.out


In [23]:
predictions = pd.read_csv(os.path.join(data_dir, 'test.csv.out'), header=None)
#predictions = [round(num) for num in predictions.squeeze().values]
predictions = [round(num) for num in predictions.squeeze().values]

print(len(test_label), 'test label')
print(len(test_data), 'test data')
print(len(train_label), 'train label')
print(len(train_data), 'train data')
print(len(valid_label), 'valid label')
print(len(valid_data), 'valid label')

# #tempData = pd.read_csv('./temp/test.csv')
# #tempData
# predictions[0] = predictions[0].str.replace('[', "")
# predictions[5] = predictions[5].str.replace(']', "")
# predictions = predictions.apply(pd.to_numeric)
# predictions = predictions.idxmax(axis=1)


predictions

1267 test label
1267 test data
10240 train label
10240 train data
1284 valid label
1284 valid label


[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0

In [24]:
from sklearn.metrics import accuracy_score
accuracy_score(test_label, predictions)

0.7308602999210734